1. 유의어 기반 증강

In [ ]:
import pandas as pd
from nltk.corpus import wordnet
import random

# CSV 파일 로드
file_path = 'train.csv'
df = pd.read_csv(file_path)

# 유의어 대체 함수
def synonym_replacement(sentence, n=1):
    words = sentence.split()
    new_words = words[:]
    random_word_list = list(set([word for word in words if wordnet.synsets(word)]))
    
    # random_word_list가 비어있지 않은지 확인
    if len(random_word_list) == 0:
        return sentence  # 동의어가 없을 경우 원래 문장 반환
    
    for _ in range(n):
        word_to_replace = random.choice(random_word_list)
        synonyms = wordnet.synsets(word_to_replace)
        synonym = random.choice(synonyms).lemmas()[0].name()
        new_words = [synonym if word == word_to_replace else word for word in new_words]
        
    return " ".join(new_words)

# 증강 데이터 생성
augmented_data = []
last_id_number = len(df)

for i in range(len(df)):
    new_sentence_1 = synonym_replacement(df.loc[i, 'sentence_1'])
    new_sentence_2 = synonym_replacement(df.loc[i, 'sentence_2'])
    
    augmented_data.append({
        'id': f"augment-{last_id_number + i}",
        'source': 'augment',
        'sentence_1': new_sentence_1,
        'sentence_2': new_sentence_2,
        'label': df.loc[i, 'label'],
        'binary-label': df.loc[i, 'binary-label']
    })

# 증강 데이터를 데이터프레임으로 변환
df_augmented = pd.DataFrame(augmented_data)

# 기존 데이터와 증강 데이터 합치기
combined_df = pd.concat([df, df_augmented], ignore_index=True)

# 합쳐진 데이터를 새로운 CSV 파일로 저장
combined_file_path = 'combined_train_augmented.csv'
combined_df.to_csv(combined_file_path, index=False)

2. 역번영 기반 데이터 증강

In [ ]:
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer

# MarianMT 모델과 토크나이저 로드
def load_model(src_lang, tgt_lang):
    model_name = f'Helsinki-NLP/opus-mt-tc-big-{src_lang}-{tgt_lang}'
    tokenizer = MarianTokenizer.from_pretrained(model_name)
    model = MarianMTModel.from_pretrained(model_name)
    return tokenizer, model

# 번역 함수
def translate_sentence(sentence, tokenizer, model):
    inputs = tokenizer(sentence, return_tensors="pt", padding=True)
    translated = model.generate(**inputs)
    return tokenizer.decode(translated[0], skip_special_tokens=True)

# 역번역 함수 (한글 -> 영어 -> 한글)
def back_translation(sentence):
    # 한글 -> 영어 모델 로드
    tokenizer_ko_en, model_ko_en = load_model('ko', 'en')
    
    # 영어 -> 한글 모델 로드
    tokenizer_en_ko, model_en_ko = load_model('en', 'ko')
    
    # 한글 -> 영어 번역
    translated_to_en = translate_sentence(sentence, tokenizer_ko_en, model_ko_en)
    
    # 영어 -> 한글 번역
    back_translated_to_ko = translate_sentence(translated_to_en, tokenizer_en_ko, model_en_ko)
    
    return back_translated_to_ko

# CSV 파일 로드
file_path = 'train.csv'
df = pd.read_csv(file_path)

# 증강 데이터 생성
augmented_data = []
last_id_number = len(df)

for i in range(len(df)):
    new_sentence_1 = back_translation(df.loc[i, 'sentence_1'])
    new_sentence_2 = back_translation(df.loc[i, 'sentence_2'])
    
    augmented_data.append({
        'id': f"augment-{last_id_number + i}",
        'source': 'augment',
        'sentence_1': new_sentence_1,
        'sentence_2': new_sentence_2,
        'label': df.loc[i, 'label'],
        'binary-label': df.loc[i, 'binary-label']
    })

# 증강 데이터를 데이터프레임으로 변환
df_augmented = pd.DataFrame(augmented_data)

# 기존 데이터와 증강 데이터 합치기
combined_df = pd.concat([df, df_augmented], ignore_index=True)

# 합쳐진 데이터를 새로운 CSV 파일로 저장
combined_file_path = 'combined_train_augmented_back_translation.csv'
combined_df.to_csv(combined_file_path, index=False)


3. BERT기반 단어 대체 증강

In [ ]:
import pandas as pd
from transformers import pipeline
import random

# CSV 파일 로드
file_path = 'train.csv'
df = pd.read_csv(file_path)

# BERT 기반 단어 대체 함수
def bert_replacement(sentence):
    fill_mask = pipeline('fill-mask', model='bert-base-uncased')
    words = sentence.split()
    if len(words) < 4:
        return sentence
    mask_idx = random.randint(0, len(words)-1)
    words[mask_idx] = '[MASK]'
    new_sentence = " ".join(words)
    result = fill_mask(new_sentence)[0]['sequence'].replace('[CLS] ', '').replace(' [SEP]', '')
    return result

# 증강 데이터 생성
augmented_data = []
last_id_number = len(df)

for i in range(len(df)):
    new_sentence_1 = bert_replacement(df.loc[i, 'sentence_1'])
    new_sentence_2 = bert_replacement(df.loc[i, 'sentence_2'])
    
    augmented_data.append({
        'id': f"augment-{last_id_number + i}",
        'source': 'augment',
        'sentence_1': new_sentence_1,
        'sentence_2': new_sentence_2,
        'label': df.loc[i, 'label'],
        'binary-label': df.loc[i, 'binary-label']
    })

# 증강 데이터를 데이터프레임으로 변환
df_augmented = pd.DataFrame(augmented_data)

# 기존 데이터와 증강 데이터 합치기
combined_df = pd.concat([df, df_augmented], ignore_index=True)

# 합쳐진 데이터를 새로운 CSV 파일로 저장
combined_file_path = 'combined_train_augmented_bert_replacement.csv'
combined_df.to_csv(combined_file_path, index=False)


4. Pegasus 기반 패러프레이징

In [ ]:
import pandas as pd
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

# CSV 파일 로드
file_path = 'train.csv'
df = pd.read_csv(file_path)

# Pegasus 기반 패러프레이징 함수
def paraphrasing(sentence):
    model_name = "tuner007/pegasus_paraphrase"
    tokenizer = PegasusTokenizer.from_pretrained(model_name)
    model = PegasusForConditionalGeneration.from_pretrained(model_name)
    
    batch = tokenizer.prepare_seq2seq_batch([sentence], truncation=True, padding='longest', return_tensors="pt")
    translated = model.generate(**batch)
    paraphrased_sentence = tokenizer.batch_decode(translated, skip_special_tokens=True)[0]
    
    return paraphrased_sentence

# 증강 데이터 생성
augmented_data = []
last_id_number = len(df)

for i in range(len(df)):
    new_sentence_1 = paraphrasing(df.loc[i, 'sentence_1'])
    new_sentence_2 = paraphrasing(df.loc[i, 'sentence_2'])
    
    augmented_data.append({
        'id': f"augment-{last_id_number + i}",
        'source': 'augment',
        'sentence_1': new_sentence_1,
        'sentence_2': new_sentence_2,
        'label': df.loc[i, 'label'],
        'binary-label': df.loc[i, 'binary-label']
    })

# 증강 데이터를 데이터프레임으로 변환
df_augmented = pd.DataFrame(augmented_data)

# 기존 데이터와 증강 데이터 합치기
combined_df = pd.concat([df, df_augmented], ignore_index=True)

# 합쳐진 데이터를 새로운 CSV 파일로 저장
combined_file_path = 'combined_train_augmented_paraphrasing.csv'
combined_df.to_csv(combined_file_path, index=False)


5. EDA 기반 랜덤 단어 삭제 증강

In [ ]:
import pandas as pd
import random

# CSV 파일 로드
file_path = 'train.csv'
df = pd.read_csv(file_path)

# EDA - 단어 랜덤 삭제 함수
def random_deletion(sentence, p=0.3):
    words = sentence.split()
    if len(words) == 1:
        return sentence
    remaining_words = [word for word in words if random.uniform(0, 1) > p]
    if len(remaining_words) == 0:
        remaining_words = [random.choice(words)]
    return " ".join(remaining_words)

# 증강 데이터 생성
augmented_data = []
last_id_number = len(df)

for i in range(len(df)):
    new_sentence_1 = random_deletion(df.loc[i, 'sentence_1'])
    new_sentence_2 = random_deletion(df.loc[i, 'sentence_2'])
    
    augmented_data.append({
        'id': f"augment-{last_id_number + i}",
        'source': 'augment',
        'sentence_1': new_sentence_1,
        'sentence_2': new_sentence_2,
        'label': df.loc[i, 'label'],
        'binary-label': df.loc[i, 'binary-label']
    })

# 증강 데이터를 데이터프레임으로 변환
df_augmented = pd.DataFrame(augmented_data)

# 기존 데이터와 증강 데이터 합치기
combined_df = pd.concat([df, df_augmented], ignore_index=True)

# 합쳐진 데이터를 새로운 CSV 파일로 저장
combined_file_path = 'combined_train_augmented_eda.csv'
combined_df.to_csv(combined_file_path, index=False)

6.  TF-IDF 기반 단어 삭제 증강

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# CSV 파일 로드
file_path = 'train.csv'
df = pd.read_csv(file_path)

# TF-IDF 기반 단어 삭제 함수
def tfidf_based_deletion(sentences, threshold=0.05):
    vectorizer = TfidfVectorizer()
    vectorizer.fit(sentences)
    
    new_sentences = []
    for sentence in sentences:
        words = sentence.split()
        tfidf_scores = vectorizer.transform([sentence]).toarray()[0]
        important_words = [words[i] for i in range(len(words)) if tfidf_scores[i] > threshold]
        if len(important_words) < 3:
            important_words = words[:3]  # 최소한 3개의 단어는 유지
        new_sentence = " ".join(important_words)
        new_sentences.append(new_sentence)
    
    return new_sentences

# 증강 데이터 생성
augmented_data = []
last_id_number = len(df)
sentences_1 = df['sentence_1'].tolist()
sentences_2 = df['sentence_2'].tolist()

new_sentences_1 = tfidf_based_deletion(sentences_1)
new_sentences_2 = tfidf_based_deletion(sentences_2)

for i in range(len(df)):
    augmented_data.append({
        'id': f"augment-{last_id_number + i}",
        'source': 'augment',
        'sentence_1': new_sentences_1[i],
        'sentence_2': new_sentences_2[i],
        'label': df.loc[i, 'label'],
        'binary-label': df.loc[i, 'binary-label']
    })

# 증강 데이터를 데이터프레임으로 변환
df_augmented = pd.DataFrame(augmented_data)

# 기존 데이터와 증강 데이터 합치기
combined_df = pd.concat([df, df_augmented], ignore_index=True)

# 합쳐진 데이터를 새로운 CSV 파일로 저장
combined_file_path = 'combined_train_augmented_tfidf_deletion.csv'
combined_df.to_csv(combined_file_path, index=False)i

7. 문장 순서 변경 데이터 증강

In [ ]:
import pandas as pd

# CSV 파일 로드
file_path = 'train.csv'
df = pd.read_csv(file_path)

# 문장 1과 문장 2의 순서를 바꿔주는 함수
def switch_sentences(sentence_1, sentence_2):
    return sentence_2, sentence_1

# 증강 데이터 생성
augmented_data = []
last_id_number = len(df)

for i in range(len(df)):
    # 문장 1과 문장 2의 순서를 바꿔서 새로운 데이터 생성
    new_sentence_1, new_sentence_2 = switch_sentences(df.loc[i, 'sentence_1'], df.loc[i, 'sentence_2'])
    
    # 증강된 데이터를 리스트에 추가
    augmented_data.append({
        'id': f"augment-{last_id_number + i}",
        'source': 'augment',
        'sentence_1': new_sentence_1,
        'sentence_2': new_sentence_2,
        'label': df.loc[i, 'label'],
        'binary-label': df.loc[i, 'binary-label']
    })

# 증강 데이터를 데이터프레임으로 변환
df_augmented = pd.DataFrame(augmented_data)

# 기존 데이터와 증강 데이터 합치기
combined_df = pd.concat([df, df_augmented], ignore_index=True)

# 합쳐진 데이터를 새로운 CSV 파일로 저장
combined_file_path = 'combined_train_augmented_switched.csv'
combined_df.to_csv(combined_file_path, index=False)


1. 데이터 삭제

In [ ]:
#삭제할 데이터 인덱스

ids_to_drop = [131, 132, 134, 146, 154, 155,
                186, 187, 202, 219, 253, 277,
                294, 311, 348, 360 ,380 ,385, 
                447, 455, 474, 481, 488, 493, 
                558, 559, 563, 564, 643, 679, 
                692, 772, 810, 881, 935, 951,
                7231, 7378, 7396, 7652, 7665,
                7690, 7898]

In [ ]:
# 1. 'id' 열에서 끝부분의 숫자를 추출하여 'id_numeric'이라는 새로운 열에 저장
df['id_numeric'] = df['id'].apply(lambda x: int(x.split('-')[-1]))

# 2. 'id_numeric' 값이 우리가 삭제하려는 ID 목록(ids_to_drop)에 포함되지 않은 행들만 남겨서 새로운 데이터프레임을 생성
df_cleaned = df[~df['id_numeric'].isin(ids_to_drop)]

# 3. 더 이상 필요 없는 'id_numeric' 열을 삭제
df_cleaned = df_cleaned.drop(columns=['id_numeric'])

# 4. 수정된 데이터프레임을 CSV 파일로 저장
cleaned_file_path_v3 = 'cleaned_augmented_train_v3.csv'
df_cleaned.to_csv(cleaned_file_path_v3, index=False)